In [3]:
from skimage import io
import numpy as np
from scipy import misc
import numpy

def divide_for_train(img, mask, size, stride):
    '''
    size:224,表示切割后的图像的大小
    stride:100，表示切割的步长
    将图像切割为大小224*224的块，图像的宽高并不都是224的倍数，剩下不够的部分被舍弃
    '''
    imgs, labels = [], []
    bands,height,width=img.shape[-1],img.shape[0],img.shape[1]
    for row in range(0, height, stride):
        for col in range(0, width, stride):
            if row + size > height or col + size > width:
                continue
            feature = []
            for band_id in range(bands):
                #feature里面apppend的大小是（224,224）
                feature.append(img[row:row + size, col:col + size,band_id])

            label = mask[row:row + size, col: col + size]
            imgs.append(feature)
            labels.append(label)

    imgs=np.array(imgs).swapaxes(1,2).swapaxes(2,3)
    labels=np.array(labels)
    return imgs, labels

def divide_for_predict(img,img_size=224):
    '''
    img的shape为:(height,weight,3),
    img_size为切割后的小图的大小，图像的高宽并不都是224的倍数，如宽为2833，2833/224=12.64，所以只有12个完整的224*224的大小的块，
    最后一个采用的是2833-224到2833的大小
    '''
    imgs = np.array([])
    height, width ,bands= img.shape
    for row in range(0, height, img_size):
        for col in range(0, width, img_size):
            if row + img_size >= height:
                row = height - img_size - 1
            if col + img_size >= width:
                col = width - img_size - 1
            temp=img[row:row + img_size, col:col + img_size,:]
#             temp=(temp/np.max(temp))*255.
            imgs = np.append(imgs, temp)
    imgs = imgs.reshape([-1, 224, 224,3])  #如果是4通道的tif图，并且需要的是4通道的数组，那么输入的img应为(height,weight,4)
    return imgs


def union(imgs, height, width,img_size=224):
    '''
    imgs的shape为(none,224,224,3)
    height:大图的高
    width:大图的宽
    将切割后的图像恢复为原图
    '''
    img = np.zeros((height, width,3))
    id = 0
    for row in range(0, height, img_size):
        for col in range(0, width, img_size):
            if row + img_size >= height:
                row = height - img_size - 1
            if col + img_size >= width:
                col = width - img_size - 1
#             print(i)
            img[row:row + img_size, col:col + img_size] = imgs[id]
            id += 1
    return img

def Test_one_img(stride=224):
    #先将图片切割为小图
    img=io.imread('./01.tif')
    imgs=divide_for_predict(img[:,:,:3])
    #将图片从小图恢复为大图，并保存
    ori_img=union(imgs[:,:,:,:3],img.shape[0],img.shape[1])
    misc.imsave('ori_img.jpg',ori_img)
    
def main(imgs_dir,masks_dir,s_imgs_dir,s_masks_dir):
    '''
    将很多图的imgs,masks进行切割并保存为图像
    '''
    imgs_list=['%02d_4bands'%i for i in range(1,2)]
    masks_list=['%02d_4bands_mask'%i for i in range(1,2)]
    for img_name,mask_name in zip(imgs_list,masks_list):
        img=io.imread('%s/%s.tif'%(imgs_dir,img_name))
        mask=io.imread('%s/%s.tif'%(masks_dir,mask_name))
        imgs,labels=divide_for_train(img,mask,224,100)
        for i in range(imgs.shape[0]):
            #只读取三个通道，将tif的4通道舍弃了一个通道,imgs.shape为：(none,224,224,4)
            a_img=imgs[i,:,:,:3]
            a_mask=labels[i,:,:]
            misc.imsave('%s/%s_%04d.jpg'%(s_imgs_dir,img_name,i),a_img)
            misc.imsave('%s/%s_%04d.png'%(s_masks_dir,mask_name,i),a_mask)

In [4]:
imgs_dir='data'
masks_dir='data'
s_imgs_dir='save'
s_masks_dir='save'

main(imgs_dir,masks_dir,s_imgs_dir,s_masks_dir)
Test_one_img()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:78: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
